<a target="_blank" href="https://colab.research.google.com/github/lukebarousse/Int_SQL_Data_Analytics_Course/blob/main/5_Views/1_View_Intro.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Views Intro

## Overview

### 🥅 Analysis Goals

Analyze cohort revenue and lifetime value (LTV) to uncover daily trends, short-term fluctuations, future potential, and long-term customer value patterns.
- **Cohort Revenue Insights:**  
  - Track cumulative revenue up to each date to measure cohort growth over time.  
  - Assess remaining cumulative revenue from each order date to analyze future revenue potential.  
  - Evaluate average LTV for each cohort using cumulative revenue and user counts, while incorporating a 7-day rolling average for recent customer activity.

### 📘 Concepts Covered

- Create views
- Views

---
## Major Topic

### 📝 Notes

- Add in specific notes

### 💻 Final Result

- Calculates the average lifetime value (LTV) for each cohort based on cumulative revenue and user count.
  - Computes a 7-day rolling average LTV for shorter timeframes to analyze recent changes in customer value.
  - Provides insights into overall customer value trends and short-term customer activity for cohorts.

#### Average and 7-Day Rolling LTV

**Views**

1. Put the query that gets the raw data the CTE called `cohort_analysis`.

In [ ]:
SELECT 
    customerkey,
    EXTRACT(YEAR FROM MIN(orderdate)) AS cohort_year, 
    orderdate,
    SUM(quantity * netprice * exchangerate) AS total_net_revenue
FROM sales
GROUP BY 
    customerkey, 
    orderdate

3. Create view in pgadmin.

4. Use the view and then calculate the total net revenue (this used to be the `cohort_summary` CTE)

In [ ]:
SELECT
    cohort_year,
    orderdate,
    SUM(total_net_revenue) AS total_revenue
FROM cohort_analysis
GROUP BY 
    cohort_year, 
    orderdate

5. Get the cumulative summary for the cohort year and order date.

In [ ]:
WITH cohort_summary AS (
    SELECT
        cohort_year,
        orderdate,
        SUM(total_net_revenue) AS total_revenue
    FROM cohort_analysis
    GROUP BY 
        cohort_year, 
        orderdate
)
    
SELECT
    cohort_year,
    orderdate,
    SUM(total_net_revenue) OVER (
        PARTITION BY cohort_year
        ORDER BY orderdate 
        ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW -- Changed 
    ) AS cumulative_revenue
FROM cohort_summary
GROUP BY 
    cohort_year, 
    orderdate

6. Put that query into a CTE called rolling_ltv.

In [ ]:
WITH cohort_summary AS (
    SELECT
        cohort_year,
        orderdate,
        SUM(total_net_revenue) AS total_revenue
    FROM cohort_analysis
    GROUP BY 
        cohort_year, 
        orderdate
),

rolling_ltv AS (
    SELECT
        cohort_year,
        orderdate,
        SUM(total_net_revenue) OVER (
            PARTITION BY cohort_year
            ORDER BY orderdate 
            ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW -- Changed 
        ) AS cumulative_revenue
    FROM cohort_summary
    GROUP BY 
        cohort_year, 
        orderdate
)

SELECT *
FROM rolling_ltv

7. Added `COUNT` to get the cumulative user count and in the main query to call specific columns.

In [ ]:
WITH cohort_summary AS (
    SELECT
        cohort_year,
        orderdate,
        SUM(total_net_revenue) AS total_revenue
    FROM cohort_analysis
    GROUP BY 
        cohort_year, 
        orderdate
),

rolling_ltv AS (
    SELECT
        cohort_year,
        orderdate,
        SUM(total_net_revenue) OVER (
            PARTITION BY cohort_year 
            ORDER BY orderdate 
            ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
        ) AS cumulative_revenue,
        COUNT(*) OVER ( -- Added 
            PARTITION BY cohort_year 
            ORDER BY orderdate ROWS 
            BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
        ) AS user_count
    FROM cohort_analysis
)

SELECT
    cohort_year,
    orderdate,
    cumulative_revenue,
    user_count
FROM rolling_ltv;

8. Caculate the rolling average LTV using: `cumulative_revenue / user_count`. 

In [ ]:
WITH cohort_summary AS (
    SELECT
        cohort_year,
        orderdate,
        SUM(total_net_revenue) AS total_revenue
    FROM cohort_analysis
    GROUP BY 
        cohort_year, 
        orderdate
),

rolling_ltv AS (
    SELECT
        cohort_year,
        orderdate,
        SUM(total_net_revenue) OVER (
            PARTITION BY cohort_year 
            ORDER BY orderdate 
            ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
        ) AS cumulative_revenue,
        COUNT(*) OVER ( -- Added 
            PARTITION BY cohort_year 
            ORDER BY orderdate ROWS 
            BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
        ) AS user_count
    FROM cohort_analysis
)

SELECT
    cohort_year,
    orderdate,
    cumulative_revenue,
    user_count,
    cumulative_revenue / user_count AS rolling_avg_ltv
FROM rolling_ltv;

9. Add two new columns to get the rolling 7 day revenue and 7 day user count

In [ ]:
%%sql

WITH cohort_summary AS (
    SELECT
        cohort_year,
        orderdate,
        SUM(total_net_revenue) AS total_revenue
    FROM cohort_analysis
    GROUP BY 
        cohort_year, 
        orderdate
),

rolling_ltv AS (
    SELECT
        cohort_year,
        orderdate,
        SUM(total_net_revenue) OVER (
            PARTITION BY cohort_year 
            ORDER BY orderdate 
            ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
        ) AS cumulative_revenue,
        COUNT(*) OVER (
            PARTITION BY cohort_year 
            ORDER BY orderdate 
            ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
        ) AS user_count,
        SUM(total_net_revenue) OVER ( -- Added 
            PARTITION BY cohort_year 
            ORDER BY orderdate
            ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
        ) AS rolling_7_day_revenue,
        COUNT(*) OVER ( -- Added 
            PARTITION BY cohort_year 
            ORDER BY orderdate 
            ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
        ) AS rolling_7_day_user_count
    FROM cohort_analysis
)

SELECT
    cohort_year,
    orderdate,
    cumulative_revenue,
    user_count,
    cumulative_revenue / user_count AS rolling_avg_ltv,
    rolling_7_day_revenue,
    rolling_7_day_user_count
FROM rolling_ltv;

10. Calulate the rolling 7 day average LTV using: `rolling_7_day_revenue / rolling_7_day_user_count`. 

In [ ]:
%%sql

WITH cohort_summary AS (
    SELECT
        cohort_year,
        orderdate,
        SUM(total_net_revenue) AS total_revenue
    FROM cohort_analysis
    GROUP BY 
        cohort_year, 
        orderdate
),

rolling_ltv AS (
    SELECT
        cohort_year,
        orderdate,
        SUM(total_net_revenue) OVER (
            PARTITION BY cohort_year 
            ORDER BY orderdate 
            ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
        ) AS cumulative_revenue,
        COUNT(*) OVER ( -- Added 
            PARTITION BY cohort_year 
            ORDER BY orderdate 
            ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
        ) AS user_count,
        SUM(total_net_revenue) OVER (
            PARTITION BY cohort_year 
            ORDER BY orderdate
            ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
        ) AS rolling_7_day_revenue,
        COUNT(*) OVER ( -- Added 
            PARTITION BY cohort_year 
            ORDER BY orderdate 
            ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
        ) AS rolling_7_day_user_count
    FROM cohort_analysis
)

SELECT
    cohort_year,
    orderdate,
    cumulative_revenue,
    user_count,
    cumulative_revenue / user_count AS rolling_avg_ltv,
    rolling_7_day_revenue / rolling_7_day_user_count AS rolling_7_avg_ltv
FROM rolling_ltv;